## Tải các thư viện cần thiết 

In [ ]:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install scikit-learn 
# !pip install seaborn

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

##  Tổng quan về dữ liệu 

In [ ]:
df = pd.read_csv("/kaggle/input/alzheimers-disease-dataset/alzheimers_disease_data.csv")

In [ ]:
df.head(5)

In [ ]:
df = df.drop(columns=["DoctorInCharge", "PatientID"])

In [ ]:
df.info()

Ta thấy dữ liệu gồm 2149 bản ghi, mỗi bản ghi gồm 33 hàng

In [ ]:
df.describe().T

In [ ]:
df["Diagnosis"].sum()

Dữ liệu gồm 760 bản ghi thuộc lớp 1, phần còn lại là lớp 0

## Tiền xử lý dữ liệu 

In [ ]:
df.isnull().sum()

Không có dữ liệu nào bị thiếu

In [ ]:
df.duplicated().sum()


Không có hai bản ghi nào trùng lặp

## Trực quan hóa dữ liệu 

In [ ]:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
axes = df.hist(figsize=(20,15), bins=30, edgecolor='black')
for ax, col in zip(axes.flatten(), numeric_columns):
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')
    ax.set_title(f'Distribution of {col}')
print("Biểu đồ cột cho thấy phân phối dữ liệu tổng quan của các trường: ")
plt.tight_layout()
plt.show()

Từ đây có thể thấy, tập dữ liệu này không quá cân bằng. Số dữ liệu chẩn đoán không mắc bệnh lớn gần gấp đôi số dữ liệu chẩn đoán có bệnh.

In [ ]:
plt.figure(figsize=(14,14))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", linewidths=0.1, annot_kws={"size": 4})  
plt.title("Ma trận tương quan giữa các chỉ số", fontsize=16)  

Từ ma trận tương quan trên ta thấy được mức độ tương quan giữa các thuộc tính là rất yếu!!!

## Sử dụng apriori để tìm những luật kết hợp


In [ ]:
def discretize_data(df):
    df_discretized = df.copy()

    # Age
    df_discretized['AgeGroup'] = pd.cut(df['Age'], bins=[59, 69, 79, 90], labels=["60-69", "70-79", "80-90"])

    # BMI
    df_discretized['BMIGroup'] = pd.cut(df['BMI'], bins=[0, 18.5, 25, 30, 100], labels=["Underweight", "Normal", "Overweight", "Obese"])

    # MMSE
    df_discretized['MMSEGroup'] = pd.cut(df['MMSE'], bins=[-1, 17, 24, 30], labels=["Severe", "Moderate", "Normal"])

    return df_discretized

In [ ]:
df_discretized = discretize_data(df)

print(df_discretized[['Age', 'AgeGroup', 'BMI', 'BMIGroup', 'MMSE', 'MMSEGroup']].head())

In [ ]:
# Encode các trường phân loại rời rạc
df_cat = df[['Gender', 'Smoking', 'FamilyHistoryAlzheimers', 'CardiovascularDisease',
       'Diabetes', 'Depression', 'HeadInjury', 'Hypertension',
       'MemoryComplaints', 'BehavioralProblems', 'Confusion',
       'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks',
       'Forgetfulness']]

df_encoded = pd.get_dummies(
    pd.concat([df_discretized[[  
        'AgeGroup', 'BMIGroup', 
        'MMSEGroup', 
    ]]], axis=1).join(df_cat),
    prefix_sep='='
)

Dựa vào các cột dữ liệu phân loại, đi khám phá những đặc trưng của người mắc bệnh.

In [ ]:
def generate_rules_1(df, cols, target_col="Diagnosis", min_sup=0.1, min_conf=0.6, max_len=2):  
    rules = []
    total = len(df)
    p_y = df[target_col].mean()

    for r in range(1, max_len + 1):  
        for antecedent in combinations(cols, r):
            condition_mask = df[list(antecedent)].all(axis=1)

            sup_x = condition_mask.mean()
            # sup_xy = df.loc[condition_mask, target_col].mean() * condition_mask.mean()
            sup_xy = condition_mask & (df[target_col] == 1)
            sup_xy = sup_xy.mean()

            if sup_x == 0:
                continue

            conf = sup_xy / sup_x

            if sup_xy >= min_sup and conf >= min_conf:
                rules.append({
                    "antecedent": antecedent,
                    "consequent": f"{target_col}=1",
                    "support": round(sup_xy, 3),
                    "confidence": round(conf, 3),
                    # "lift": round(lift, 3)
                })

    return pd.DataFrame(rules).sort_values(by="confidence", ascending=False)

In [ ]:
def generate_rules_0(df, cols, target_col="Diagnosis", min_sup=0.1, min_conf=0.6, max_len=2):
    rules = []
    total = len(df)
    
    p_y = 1 - df[target_col].mean()  

    for r in range(1, max_len + 1):
        for antecedent in combinations(cols, r):
            condition_mask = df[list(antecedent)].all(axis=1)
            
            sup_x = condition_mask.mean()
            
            if sup_x == 0:  
                continue
            
            sup_xy = condition_mask & (df[target_col] == 0)
            # print(f'supxy = {sup_xy}')
            sup_xy = sup_xy.mean()

            conf = sup_xy / sup_x if sup_x > 0 else 0

            if sup_xy >= min_sup and conf >= min_conf:
                rules.append({
                    "antecedent": list(antecedent),
                    "consequent": f"{target_col}=0",
                    "support": round(sup_xy, 3),
                    "confidence": round(conf, 3),
                })
    
    return pd.DataFrame(rules).sort_values(by="confidence", ascending=False)

In [ ]:
from itertools import combinations
rules_df1 = generate_rules_1(
    df_encoded.join(df[['Diagnosis']]),
    cols=df_encoded.columns.tolist(),
    min_sup=0.05,
    min_conf=0.6,
    max_len= 3
)

rules_df2 = generate_rules_0(
    df_encoded.join(df[['Diagnosis']]),
    cols=df_encoded.columns.tolist(),
    min_sup=0.05,
    min_conf=0.9,
    max_len= 3
)

In [ ]:
rules_df1

In [ ]:
rules_df2

## Chia dữ liệu để chạy mô hình dự đoán 

In [ ]:
x = df.drop(columns=["Diagnosis"])
y = df["Diagnosis"]
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [ ]:
x_test

In [ ]:
y_test

## MODEL dự đoán

### Thêm các thư viện cần thiết


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.metrics import classification_report, confusion_matrix, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB


best_models = {}

#### Hàm đánh giá giải thuật 

In [ ]:
def evaluate_model(grid):
    best_model = grid.best_estimator_
    print(" Best Parameters:", grid.best_params_)

    y_pred = best_model.predict(x_test)

    print("\n Classification Report:")
    print(classification_report(y_test, y_pred))

    print("\n Confusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

    weighted_f1 = f1_score(y_test, y_pred, average='weighted')
    final_score = weighted_f1 * 100

    print(f'Weighted F1-Score: {final_score:.2f}')
    


### Chúng em sử dụng các thuật toán sau để đánh dự đoán: ...

### Chi tiết các thuật toán

#### SVM

In [ ]:
categorical_cols = ['Gender', 'Smoking', 
    'FamilyHistoryAlzheimers', 'CardiovascularDisease',
    'Diabetes', 'Depression', 'HeadInjury', 'Hypertension',
    'MemoryComplaints', 'BehavioralProblems', 'Confusion',
    'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks',
    'Forgetfulness']
numerical_cols = [col for col in x.columns if col not in categorical_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)


In [ ]:
model_pipeline_svc = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classifier', SVC())
])

param_grid = {
    'classifier__C': [0.1, 1, 10],
    'classifier__gamma': ['scale', 'auto'],
    'classifier__kernel': ['rbf', 'linear']
}
grid_search = GridSearchCV(model_pipeline_svc, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_
evaluate_model(grid_search)

SVM không phù hợp có thể là do có quá nhiều dữ liệu phân loại, khi one hot làm tăng số chiều và rời rạc dữ liệu .Cũng có thể do dữ liệu phi tuyến. SVM dựa trên khoảng cách, việc onehot có thể làm các dữ liệu mất đi ý nghĩa về khoảng cách. Hiểu đơn giản là khi tăng số chiều thì làm tăng khoảng cách, làm ảnh hưởng đến các đặc trưng khác.

#### Naive Bayes

In [ ]:
model_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classifier', GaussianNB())
])

param_grid = {
    'classifier__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

grid_search = GridSearchCV(model_pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_

evaluate_model(grid_search)

#### DecisionTree 

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)

param_grid = {
    'criterion': ['entropy'], 
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 15, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2'],
    'class_weight': [None, 'balanced']
}


dt_grid = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
best_models['DecisionTree'] = dt_grid

dt_grid.fit(x_train, y_train)

best_model = dt_grid.best_estimator_

evaluate_model(dt_grid)

#### RandomForest

In [ ]:
rf_model = RandomForestClassifier(random_state=42)

param_grid_rf = {
    'n_estimators': [50, 100],                      
    'criterion': ['entropy'],      
    'max_depth': [None, 10, 20, 30],                     
    'min_samples_split': [2, 5, 10],                    
    'min_samples_leaf': [1, 2, 4],          
    'bootstrap': [True, False],                          
    'class_weight': [None, 'balanced']              
}

grid_rf = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid_rf,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
best_models['RandomForest'] = grid_rf

grid_rf.fit(x_train, y_train)

evaluate_model(grid_rf)

#### XGBoost

In [ ]:
xgb_model = XGBClassifier(
    tree_method='hist',  
    gpu_id=1, 
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)

param_grid_xgb = {
    'n_estimators': [50, 100],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'reg_lambda': [0.01, 0.1, 1],
    'reg_alpha': [0, 0.1, 0.5]
}

random_search_xgb = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid_xgb,
    n_iter=100,
    scoring='accuracy',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)
best_models['XGBoost'] = random_search_xgb

random_search_xgb.fit(x_train, y_train)

evaluate_model(random_search_xgb)

#### LightGBM

In [ ]:
lgbm_model = LGBMClassifier(
    device='gpu',     
    boosting_type='gbdt',
    random_state=42
)

param_grid_lgbm = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 10, -1],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_lambda': [0.01, 0.1, 1],
    'reg_alpha': [0, 0.1, 0.5]
}

random_search_lgbm = RandomizedSearchCV(
    estimator=lgbm_model,
    param_distributions=param_grid_lgbm,
    n_iter=100,
    scoring='accuracy',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)
best_models['LightGBM'] = random_search_lgbm


random_search_lgbm.fit(x_train, y_train)

evaluate_model(random_search_lgbm)

#### CatBoost

In [ ]:
catboost_model = CatBoostClassifier(
    task_type='GPU',          
    devices='0',              
    verbose=0,                
    random_state=42
)

# Grid tham số phù hợp với CatBoost
param_grid_catboost = {
    'iterations': [100, 200, 300],
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'bagging_temperature': [0, 1, 2, 5],
    'border_count': [32, 64, 128],  
    'random_strength': [1, 5, 10]
}

random_search_catboost = RandomizedSearchCV(
    estimator=catboost_model,
    param_distributions=param_grid_catboost,
    n_iter=100,            
    scoring='accuracy',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=1
)
best_models['CatBoost'] = random_search_catboost

random_search_catboost.fit(x_train, y_train)

evaluate_model(random_search_catboost)

In [ ]:
# # Tạo các pipeline cho các thuật toán
# models = {
#     "LogisticRegression": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', LogisticRegression(max_iter=1000))
#     ]),
#     "KNN": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', KNeighborsClassifier())
#     ]),
#     "SVM": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', SVC())
#     ]),
#     "DecisionTree": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', DecisionTreeClassifier())
#     ]),
#     "RandomForest": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', RandomForestClassifier())
#     ]),
#     "NaiveBayes": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', GaussianNB())
#     ]),
#     "XGBoost": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
#     ]),
#     "LightGBM": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', LGBMClassifier())
#     ]),
#     "CatBoost": Pipeline([
#         ('scaler', StandardScaler()),
#         ('classifier', CatBoostClassifier(verbose=0))
#     ]),
# }

# # Grid Search cho các thuật toán có thể tinh chỉnh
# param_grids = {
#     "LogisticRegression": {
#         'classifier__C': [0.1, 1.0, 10],
#         'classifier__solver': ['lbfgs', 'liblinear']
#     },
#     "KNN": {
#         'classifier__n_neighbors': [3, 5, 7]
#     },
#     "SVM": {
#         'classifier__C': [0.1, 1, 10],
#         'classifier__kernel': ['linear', 'rbf']
#     },
#     "DecisionTree": {
#         'classifier__criterion': ['entropy'],
#         'classifier__max_depth': [None, 10, 20],
#         'classifier__min_samples_split': [2, 5]
#     },
#     "RandomForest": {
#         'classifier__n_estimators': [50, 100],
#         'classifier__max_depth': [None, 10]
#     },
#     "XGBoost": {
#         'classifier__n_estimators': [50, 100],
#         'classifier__learning_rate': [0.05, 0.1]
#     },
#     "LightGBM": {
#         'classifier__n_estimators': [50, 100],
#         'classifier__learning_rate': [0.05, 0.1]
#     },
#     "CatBoost": {
#         'classifier__iterations': [100, 200],
#         'classifier__learning_rate': [0.05, 0.1]
#     }
# }

# # Huấn luyện từng mô hình
# for model_name, model in models.items():
#     print(f"\nTraining {model_name}...")

#     if model_name in param_grids:
#         grid = GridSearchCV(model, param_grids[model_name], cv=5, scoring='accuracy', n_jobs=-1)
#         grid.fit(x_train, y_train)
#         best_models[model_name] = grid.best_estimator_
#         print(f"Best {model_name} Parameters: {grid.best_params_}")
#     else:
#         model.fit(x_train, y_train)
#         best_models[model_name] = model

#     # Dự đoán và đánh giá
#     y_pred = best_models[model_name].predict(x_test)
#     print(f"\n{model_name} Classification Report:")
#     print(classification_report(y_test, y_pred))

## Đánh giá 

In [ ]:
from sklearn.metrics import confusion_matrix
fig, axes = plt.subplots(3, 3, figsize=(18, 18)) 
axes = axes.flatten()  
model_names = list(best_models.keys())

for i, model_name in enumerate(model_names):
    y_pred = best_models[model_name].predict(x_test)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f"{model_name} Confusion Matrix")
    axes[i].set_xlabel("Predicted")
    axes[i].set_ylabel("Actual")

for j in range(len(model_names), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))

for model_name in model_names:
    model = best_models[model_name]

    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(x_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{model_name} (AUC = {roc_auc:.2f})")
    else:
        print(f"Bỏ qua {model_name} vì không hỗ trợ predict_proba.")

plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for Different Models")
plt.legend()
plt.show()
